<img src='https://raw.githubusercontent.com/alexisuaguilaru/WebScraping_CIAFactbook/refs/heads/main/Resources/Proyecto03_DataWrangling_AlexisAguilar.png' alt='cover'>

# Introduction

The following notebook aims to develop the different processes and decisions taken to perform the Data Wrangling on the dataset obtained from Web Scraping. The first step is to deal with the missing values of the different attributes based on the knowledge acquired from the Web Scraping itself and the meaning of the attributes. With these modifications the final dataset that will be used in the Exploratory Data Analysis is generated.

Where the fundamental part of this notebook is found in [Data Wrangling](#2-data-wrangling) where the specific elements (countries) that do not have a specific attribute are also exposed, in addition to mentioning the value with which the missing values are imputed.

# 0. Import Libraries

In [1]:
# Import Libraries

import pandas as pd

In [2]:
# Other code 

import SourceDataWrangling as src

PATH_DATASET = '../Dataset/'
RANDOM_STATE = 8013

# 1. Load Dataset

First the dataset obtained from the Web Scraping process is loaded, in order to determine missing or null values in the different attributes.

Based on [Requirements](../RequirementsDocument.pdf), the attributes are renamed to match those specified and those relevant to the study are retained.

By performing the Web Scraping was acquired the knowledge to determine why several of the attributes have null values. In all cases it is related to the fact that the field is empty or its value is `0` or some other specific value.

In [3]:
WorldFactbook_Dataset_Raw = pd.read_csv(PATH_DATASET+'Dataset_Raw.csv')

_features_renaming = {
    'country_name' : 'name',
    'real_gdp_purchasing_power_parity' : 'gdp',
    'unemployment_rate' : 'unemployment',
    'taxes_and_other_revenues' : 'taxes',
    'debt_external' : 'debt',
}

Features = [
    'name',
    'area',
    'population',
    'gdp',
    'unemployment',
    'taxes',
    'debt',
    'exchange_rates',
    'internet_users',
    'internet_percent',
    'airports',
    'merchant_marine',
    'military_expenditures',
    'image_urls',
]

WorldFactbook_Dataset_Raw_1 = WorldFactbook_Dataset_Raw.rename(columns=_features_renaming)[Features]

## Count of Null Values by Feature

In [4]:
_null_values_feature = WorldFactbook_Dataset_Raw_1.isnull().sum()

_null_values_feature

name                       0
area                       0
population                15
gdp                       28
unemployment              50
taxes                     47
debt                     132
exchange_rates            15
internet_users            19
internet_percent          18
airports                  18
merchant_marine           66
military_expenditures     85
image_urls                 5
dtype: int64

## Examples of Countries

In [5]:
_sample_countries = WorldFactbook_Dataset_Raw_1.sample(5,random_state=RANDOM_STATE)

_sample_countries

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
232,Turkmenistan,488100.0,5744151.0,1.315760e+11,4.3,14.9,3.696000e+09,4.125,120627171.0,21.0,23.0,73.0,1.9,https://www.cia.gov/the-world-factbook/static/...
24,Benin,112622.0,14697052.0,5.251000e+10,1.7,17.1,6.309000e+09,606.345,470305664.0,32.0,10.0,6.0,0.7,https://www.cia.gov/the-world-factbook/static/...
202,Sierra Leone,71740.0,9121049.0,2.567100e+10,3.1,15.6,1.451000e+09,21.305,191542029.0,21.0,8.0,584.0,0.6,https://www.cia.gov/the-world-factbook/static/...
85,The Gambia,11300.0,2523327.0,7.911000e+09,6.5,20.3,9.024210e+08,61.096,116073042.0,46.0,1.0,15.0,0.6,https://www.cia.gov/the-world-factbook/static/...
133,Libya,1759540.0,7361263.0,9.116100e+10,18.6,51.6,NaN,4.832,655152407.0,89.0,75.0,96.0,NaN,https://www.cia.gov/the-world-factbook/static/...


As shown, many of the attributes have missing values, which can be explained by using the knowledge acquired during Web Scraping. And as explained below some of these values can be properly imputed.

# 2. Data Wrangling

For each feature with missing or null values, it is determined, by means of what that feature represents, which value will be useb to fill the missing values.

## 2.1 Drop `'World'` Entry

During Web Scraping, it was detected that global data (World) was extracted, so it will first be eliminated because it is not of interest for the study.

In [6]:
# Keeping all the countries except 'World'

WorldFactbook_Dataset_Raw_2 = WorldFactbook_Dataset_Raw_1.query("name != 'World'")

WorldFactbook_Dataset_Raw_2.reset_index(drop=True,inplace=True)

## 2.2 `population` Feature

Of the countries or entries that do not have `population` values, in most cases they are uninhabited islands or places with temporary inhabitants. Therefore, in both cases there are no inhabitants (population is zero).

In [7]:
# Exploring countries without `population`

WorldFactbook_Dataset_Raw_2.query("population != population")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.00,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
32,British Indian Ocean Territory,60.00,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
49,Clipperton Island,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
56,Coral Sea Islands,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
83,French Southern and Antarctic Lands,55.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
101,Heard Island and McDonald Islands,412.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
116,Jan Mayen,377.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...


In [8]:
# Filling missing values on `population` with 0

WorldFactbook_Dataset_Raw_2__population = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'population',0)

## 2.3 `gdp` Feature

Due to the importance that this feature represents for the study, it is necessary to be careful about the value assigned for imputation, but as many of the missing values are for islands, protected areas and oceans, the best value is `0`.

In addition to all this, it should be considered that these islands or territories are incorporated to a country, so the value of its `GDP` could be equal to that of the country to which it belongs.

In [9]:
# Exploring countries without `gdp`

WorldFactbook_Dataset_Raw_2.query("gdp != gdp")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.00,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
7,Anguilla,91.00,19416.0,NaN,NaN,46.7,NaN,2.700,1584346.0,81.6,1.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
32,British Indian Ocean Territory,60.00,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
48,Christmas Island,135.00,1692.0,NaN,NaN,NaN,NaN,1.505,132991.0,78.6,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
49,Clipperton Island,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
50,Cocos (Keeling) Islands,14.00,593.0,NaN,NaN,NaN,NaN,1.505,7946.0,13.4,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
56,Coral Sea Islands,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...


In [10]:
# Filling missing values on `gdp` with 0

WorldFactbook_Dataset_Raw_2__gdp = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'gdp',0)

## 2.4 `unemployment` Feature

In several of the missing data are from non-productive islands or there is not enough data to determine the unemployment rate; therefore, under these considerations, the missing values can be imputed with `0`, although the mean could be used to indicate that they behave as the average unemployment rate of the countries.

In [11]:
# Exploring countries without `unemployment`

WorldFactbook_Dataset_Raw_2.query("unemployment != unemployment")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.00,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
4,American Samoa,224.00,43895.0,6.580000e+08,NaN,37.8,NaN,1.000,1768968.0,40.3,3.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
7,Anguilla,91.00,19416.0,NaN,NaN,46.7,NaN,2.700,1584346.0,81.6,1.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
9,Antigua and Barbuda,443.00,102634.0,2.703000e+09,NaN,19.6,NaN,2.700,8005452.0,78.0,4.0,614.0,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
32,British Indian Ocean Territory,60.00,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
33,British Virgin Islands,151.00,40102.0,5.000000e+08,NaN,38.9,NaN,1.000,3127956.0,78.0,3.0,29.0,NaN,https://www.cia.gov/the-world-factbook/static/...
43,Cayman Islands,264.00,66653.0,5.705000e+09,NaN,38.9,NaN,0.833,5398893.0,81.0,3.0,130.0,NaN,https://www.cia.gov/the-world-factbook/static/...


In [12]:
# Filling missing values on `unemployment` with 0

WorldFactbook_Dataset_Raw_2__unemployment = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'unemployment',0)

## 2.5 `taxes` Feature

Although many of the missing values belong to islands, taxes still apply, so these missing values in `taxes` should be imputed with the average of all countries. This is done to represent that they have a behavior similar to the global average.

In [13]:
# Exploring countries without `taxes`

WorldFactbook_Dataset_Raw_2.query("taxes != taxes")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.00,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
32,British Indian Ocean Territory,60.00,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
48,Christmas Island,135.00,1692.0,NaN,NaN,NaN,NaN,1.505,132991.0,78.6,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
49,Clipperton Island,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
50,Cocos (Keeling) Islands,14.00,593.0,NaN,NaN,NaN,NaN,1.505,7946.0,13.4,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
55,Cook Islands,236.00,7761.0,2.660000e+08,NaN,NaN,NaN,1.628,502913.0,64.8,10.0,190.0,NaN,https://www.cia.gov/the-world-factbook/static/...
56,Coral Sea Islands,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...


In [14]:
# Filling missing values on `taxes` with its mean

_mean_taxes = WorldFactbook_Dataset_Raw_2['taxes'].mean()
WorldFactbook_Dataset_Raw_2__taxes = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'taxes',_mean_taxes)

## 2.6 `debt` Feature

Since there are many missing values in this feature, in addition to the fact that not all countries are islands or natural areas, this imputation becomes delicate.

A candidate value for this imputation is to use the median of `debt`, because the median allows to indicate that these countries with missing values will have a behavior similar to `50%` of the countries with `debt`, that is, it is assumed that their debt is at most the median of `debt`.

In [15]:
# Exploring countries without `debt`

WorldFactbook_Dataset_Raw_2.query("debt != debt")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.0,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
4,American Samoa,224.0,43895.0,6.580000e+08,NaN,37.8,NaN,1.000,1.768968e+06,40.3,3.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
5,Andorra,468.0,85370.0,5.226000e+09,3.7,69.0,NaN,0.924,8.110150e+06,95.0,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
7,Anguilla,91.0,19416.0,NaN,NaN,46.7,NaN,2.700,1.584346e+06,81.6,1.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,Venezuela,912050.0,31250306.0,2.690680e+11,5.5,44.2,NaN,9.975,1.937519e+09,62.0,509.0,272.0,0.6,https://www.cia.gov/the-world-factbook/static/...
246,Virgin Islands,1910.0,104377.0,4.900000e+09,12.0,28.9,NaN,1.000,6.680128e+06,64.0,2.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
247,West Bank,5860.0,3243369.0,2.744600e+10,24.4,21.5,NaN,3.360,2.821731e+08,87.0,1.0,NaN,NaN,NaN
248,Wake Island,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...


In [16]:
# Filling missing values on `debt` with its mean

_median_debt = WorldFactbook_Dataset_Raw_2['debt'].median()
WorldFactbook_Dataset_Raw_2__debt = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'debt',_median_debt)

## 2.7 `exchange_rates` Feature

In the countries that do not have this feature, two cases are detected: either they are islands belonging to a country or they do not have a national currency. Therefore, a manual imputation is performed for the missing values with the values of the corresponding countries and in the case that they do not have a national currency, it is left as `1`.

In [17]:
# Exploring countries without `exchange_rates`

WorldFactbook_Dataset_Raw_2.query("exchange_rates != exchange_rates")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
8,Antarctica,14200000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
49,Clipperton Island,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
56,Coral Sea Islands,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
83,French Southern and Antarctic Lands,55.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
101,Heard Island and McDonald Islands,412.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
116,Jan Mayen,377.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
158,Navassa Island,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
172,Paracel Islands,8.00,2230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [18]:
# Manual filling missing values on `exchange_rates`

_index_missing__exchange_rates = WorldFactbook_Dataset_Raw_2.query("exchange_rates != exchange_rates").index
_filling_values__exchange_rates = [1,1.515,
                                   10.746,0.924,
                                   1.515,0.924,
                                   1.515,10.746,
                                   1.369,1,
                                   1,0.782,
                                   1.369,1.369]

WorldFactbook_Dataset_Raw_2__exchange_rates = WorldFactbook_Dataset_Raw_2['exchange_rates'].copy()
WorldFactbook_Dataset_Raw_2__exchange_rates.iloc[_index_missing__exchange_rates] = _filling_values__exchange_rates

## 2.8 `internet_users` Feature

In most cases these are uninhabited islands, protected areas or public information, so the `internet_users` feature does not have a value, i.e. it is `0`.

In [19]:
# Exploring countries without `internet_users`

WorldFactbook_Dataset_Raw_2.query("internet_users != internet_users")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.00,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
32,British Indian Ocean Territory,60.00,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
49,Clipperton Island,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
56,Coral Sea Islands,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
83,French Southern and Antarctic Lands,55.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
101,Heard Island and McDonald Islands,412.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
112,Isle of Man,572.00,92269.0,6.792000e+09,1.1,NaN,NaN,0.782,NaN,NaN,4.0,269.0,NaN,https://www.cia.gov/the-world-factbook/static/...


In [20]:
# Filling missing values on `internet_users` with 0

WorldFactbook_Dataset_Raw_2__internet_users = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'internet_users',0)

## 2.9 `internet_percent` Feature

As in `internet_users`, missing values in `internet_percent` are imputed with `0`.

In [21]:
# Exploring countries without `internet_percent`

WorldFactbook_Dataset_Raw_2.query("internet_percent != internet_percent")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.00,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
32,British Indian Ocean Territory,60.00,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
49,Clipperton Island,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
56,Coral Sea Islands,3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
83,French Southern and Antarctic Lands,55.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
101,Heard Island and McDonald Islands,412.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
112,Isle of Man,572.00,92269.0,6.792000e+09,1.1,NaN,NaN,0.782,NaN,NaN,4.0,269.0,NaN,https://www.cia.gov/the-world-factbook/static/...
116,Jan Mayen,377.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...


In [22]:
# Filling missing values on `internet_percent` with 0

WorldFactbook_Dataset_Raw_2__internet_percent = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'internet_percent',0)

## 2.10 `airports` Feature

Of the countries with missing values in `airports`, most are uninhabited islands or countries that have access to this infrastructure using other means. Therefore, missing values are imputed with `0`.

In [23]:
# Exploring countries without `airports`

WorldFactbook_Dataset_Raw_2.query("airports != airports")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.0,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
5,Andorra,468.0,85370.0,5.226000e+09,3.7,69.0,NaN,0.924,8110150.0,95.0,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
13,Ashmore and Cartier Islands,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
30,Bouvet Island,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
49,Clipperton Island,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
56,Coral Sea Islands,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
86,"Gaza, Gaza Strip",360.0,2141643.0,2.744600e+10,24.4,21.5,NaN,3.360,186322941.0,87.0,NaN,NaN,NaN,NaN
101,Heard Island and McDonald Islands,412.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
102,Holy See (Vatican City),0.0,1000.0,NaN,NaN,NaN,NaN,0.925,87000.0,87.0,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
116,Jan Mayen,377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...


In [24]:
# Filling missing values on `airports` with 0

WorldFactbook_Dataset_Raw_2__airports = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'airports',0)

## 2.11 `merchant_marine` Feature

The fact that they do not have the `merchant_marine` attribute can be related to the fact that they are landlocked countries or regions, countries that do not have port infrastructure or are uninhabited islands. Therefore, in these three cases, `merchant_marine` is imputed with `0`.

In [25]:
# Exploring countries without `merchant_marine`

WorldFactbook_Dataset_Raw_2.query("merchant_marine != merchant_marine")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
0,Afghanistan,652230.00,40121552.0,8.259500e+10,13.3,9.9,2.717000e+09,76.814,7.221879e+08,18.0,68.0,NaN,3.3,https://www.cia.gov/the-world-factbook/static/...
1,Akrotiri and Dhekelia,254.00,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
4,American Samoa,224.00,43895.0,6.580000e+08,NaN,37.8,NaN,1.000,1.768968e+06,40.3,3.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
5,Andorra,468.00,85370.0,5.226000e+09,3.7,69.0,NaN,0.924,8.110150e+06,95.0,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,"Baker Island, Howland Island, Jarvis Island, J...",6959.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
242,Uzbekistan,447400.00,36520593.0,3.567970e+11,4.5,11.6,2.571400e+10,12652.287,3.250333e+09,89.0,74.0,NaN,2.8,https://www.cia.gov/the-world-factbook/static/...
247,West Bank,5860.00,3243369.0,2.744600e+10,24.4,21.5,NaN,3.360,2.821731e+08,87.0,1.0,NaN,NaN,NaN
248,Wake Island,7.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...


In [26]:
# Filling missing values on `merchant_marine` with 0

WorldFactbook_Dataset_Raw_2__merchant_marine = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'merchant_marine',0)

## 2.12 `military_expenditures` Feature

Countries that do not have `military_expenditures` verify that they are islands, are not publicly available their data or do not have military expenditures. Although some of the countries do have military expenditures (although they are not publicly reported), the best strategy is to impute the missing values with the global average of `military_expenditures`; this is to indicate that they have a similar trend to the global behavior.

In [27]:
# Exploring countries without `military_expenditures`

WorldFactbook_Dataset_Raw_2.query("military_expenditures != military_expenditures")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
1,Akrotiri and Dhekelia,254.0,NaN,NaN,NaN,NaN,NaN,0.925,NaN,NaN,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
4,American Samoa,224.0,43895.0,6.580000e+08,NaN,37.8,NaN,1.000,1768968.0,40.3,3.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
5,Andorra,468.0,85370.0,5.226000e+09,3.7,69.0,NaN,0.924,8110150.0,95.0,NaN,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
7,Anguilla,91.0,19416.0,NaN,NaN,46.7,NaN,2.700,1584346.0,81.6,1.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
8,Antarctica,14200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,31.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Virgin Islands,1910.0,104377.0,4.900000e+09,12.0,28.9,NaN,1.000,6680128.0,64.0,2.0,2.0,NaN,https://www.cia.gov/the-world-factbook/static/...
247,West Bank,5860.0,3243369.0,2.744600e+10,24.4,21.5,NaN,3.360,282173103.0,87.0,1.0,NaN,NaN,NaN
248,Wake Island,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,https://www.cia.gov/the-world-factbook/static/...
249,Wallis and Futuna,142.0,15964.0,NaN,NaN,NaN,NaN,110.347,731151.0,45.8,2.0,1.0,NaN,https://www.cia.gov/the-world-factbook/static/...


In [28]:
# Filling missing values on `military_expenditures` with its mean

_median_military_expenditures = WorldFactbook_Dataset_Raw_2['military_expenditures'].median()
WorldFactbook_Dataset_Raw_2__military_expenditures = src.FillMissingValues(WorldFactbook_Dataset_Raw_2,'military_expenditures',_median_military_expenditures)

## 2.13 `image_urls` Feature

Of the countries that do not have the image of their flag, it is because the page [CIA: The World Factbook](https://www.cia.gov/the-world-factbook/) does not have the representative flag of those countries.

In [29]:
# Exploring countries without `image_urls`

WorldFactbook_Dataset_Raw_2.query("image_urls != image_urls")

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls
86,"Gaza, Gaza Strip",360.0,2141643.0,2.744600e+10,24.4,21.5,NaN,3.36,186322941.0,87.0,NaN,NaN,NaN,NaN
172,Paracel Islands,8.0,2230.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
212,Spratly Islands,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,West Bank,5860.0,3243369.0,2.744600e+10,24.4,21.5,NaN,3.36,282173103.0,87.0,1.0,NaN,NaN,NaN


# 3. Dump Modified Dataset

Once the missing values have been processed, manual adjustments are made on `name` for some special cases on the names. Afterwards, Data Wrangling is applied to the final dataset.

Finally, the `gdp` values are encoded in categorical values following the conventions established in [Requirements](../RequirementsDocument.pdf).

And with this last modification, the dataset is saved with the last modifications.

## 3.1 Renaming Country Names

The names of the following entries are renamed using the consequent names:

* Svalbard (sometimes referred to as Spitsbergen, the largest island in the archipelago) $\to$ Svalbard

* Baker Island, Howland Island, Jarvis Island, Johnston Atoll, Kingman Reef, Midway Islands, Palmyra Atoll $\to$ United States Pacific Island Wildlife Refuges

In [30]:
# Renaming some values in `name`

WorldFactbook_Dataset_Raw_3 = WorldFactbook_Dataset_Raw_2.copy()

_index_long_names = [217,240]
_rename_long_names = ['Svalbard','United States Pacific Island Wildlife Refuges']

WorldFactbook_Dataset_Raw_3.loc[_index_long_names,'name'] = _rename_long_names

## 3.2 Merge Individual Data Wrangling

From the different considerations and imputation values to perform the [Data Wrangling](#2-data-wrangling), they are applied to the final dataset, and it is checked that it has no missing values except in `image_urls`.

As the last point is verified, the Data Wrangling has been properly performed on the dataset.

In [31]:
# Applying Data Wrangling 

WorldFactbook_Dataset_Raw_3_Wrangling = WorldFactbook_Dataset_Raw_3.copy()

DataWrangling_features = [
    WorldFactbook_Dataset_Raw_2__population,
    WorldFactbook_Dataset_Raw_2__gdp,
    WorldFactbook_Dataset_Raw_2__unemployment,
    WorldFactbook_Dataset_Raw_2__taxes,
    WorldFactbook_Dataset_Raw_2__debt,
    WorldFactbook_Dataset_Raw_2__exchange_rates,
    WorldFactbook_Dataset_Raw_2__internet_users,
    WorldFactbook_Dataset_Raw_2__internet_percent,
    WorldFactbook_Dataset_Raw_2__airports,
    WorldFactbook_Dataset_Raw_2__merchant_marine ,
    WorldFactbook_Dataset_Raw_2__military_expenditures,
]

for _data_wrangling_feature in DataWrangling_features:
    _feature = _data_wrangling_feature.name
    WorldFactbook_Dataset_Raw_3_Wrangling[_feature] = _data_wrangling_feature

In [32]:
# Count of Null Values by Feature

WorldFactbook_Dataset_Raw_3_Wrangling.isnull().sum()

name                     0
area                     0
population               0
gdp                      0
unemployment             0
taxes                    0
debt                     0
exchange_rates           0
internet_users           0
internet_percent         0
airports                 0
merchant_marine          0
military_expenditures    0
image_urls               4
dtype: int64

## 3.3 Encode `gdp` Feature

Following the conventions and notation in [Requirements](../RequirementsDocument.pdf), the `gdp_encode` feature is generated based on `gdp` with the following rules:

* `low-income` $\implies$ `gdp` $\le 1.0$ trillions

* `average-income` $\implies 1.0 <$ `gdp` $\le 3.0$ trillions

* `high-income` $\implies$ `gdp` $3.0 >$ trillions

In [33]:
# Encoding `gdp`

WorldFactbook_Dataset = WorldFactbook_Dataset_Raw_3_Wrangling.copy()

WorldFactbook_Dataset['gdp_encode'] = src.EncoderGDP(WorldFactbook_Dataset)


# Examples of Processed Countries

_sample_countries = WorldFactbook_Dataset.sample(5,random_state=RANDOM_STATE)

_sample_countries

,name,area,population,gdp,unemployment,taxes,debt,exchange_rates,internet_users,internet_percent,airports,merchant_marine,military_expenditures,image_urls,gdp_encode
232,Turkmenistan,488100.0,5744151.0,1.315760e+11,4.3,14.9,3.696000e+09,4.125,120627171.0,21.0,23.0,73.0,1.90,https://www.cia.gov/the-world-factbook/static/...,low-income
24,Benin,112622.0,14697052.0,5.251000e+10,1.7,17.1,6.309000e+09,606.345,470305664.0,32.0,10.0,6.0,0.70,https://www.cia.gov/the-world-factbook/static/...,low-income
202,Sierra Leone,71740.0,9121049.0,2.567100e+10,3.1,15.6,1.451000e+09,21.305,191542029.0,21.0,8.0,584.0,0.60,https://www.cia.gov/the-world-factbook/static/...,low-income
85,The Gambia,11300.0,2523327.0,7.911000e+09,6.5,20.3,9.024210e+08,61.096,116073042.0,46.0,1.0,15.0,0.60,https://www.cia.gov/the-world-factbook/static/...,low-income
133,Libya,1759540.0,7361263.0,9.116100e+10,18.6,51.6,6.712000e+09,4.832,655152407.0,89.0,75.0,96.0,1.65,https://www.cia.gov/the-world-factbook/static/...,low-income


## 3.4 Dump Dataset

With the last modifications made, it proceeds with saving the dataset without missing values and with the pertinent modifications.

In [ ]:
# Dumping dataset 

WorldFactbook_Dataset.to_csv(PATH_DATASET+'Dataset.csv',index=False)

# Conclusions

After carrying out different projects where Data Wrangling is applied as part of the data cleaning process, it is clear that this stage allows to collect many of these ideas and integrate new ones based on the knowledge of the problem we are working on. With this, it is necessary to know how to use domain knowledge to improve the accuracy of data cleaning.

# References

* [1] [Proyecto 3: *Web Scraping con Scrapy*](../RequirementsDocument.pdf). Tinoco Martinez Sergio Rogelio
* [2] DataFrame. Pandas. https://pandas.pydata.org/docs/reference/frame.html
* [3] API Reference. Numpy. https://numpy.org/doc/stable/reference/index.html
* [4] API Reference. Marimo. https://docs.marimo.io/api/